In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import random
import datetime as dt
import matplotlib.pyplot as plt
# import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import gc
# del a
# gc.collect()

from matplotlib.font_manager import FontProperties
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from tqdm import tqdm

fontproperties = FontProperties(fname = 'font_ch.ttf', size = 14)
font = 'font_ch.ttf'




In [2]:
###columns
inventory_cols = ['day','item_id','warehouse_id','warehouse_city_id','total_begin_qty','total_end_qty','Replen_in_qty',
                  'transfer_in_qty','sale_out_qty','transfer_out_qty']
item_cols = ['date','item_id','front_page_item_id','merchant_id','brand_id','category_id','sub_category_id','pc_pv',
             'app_pv','pc_uv','app_uv','if_cainiao']
logistic_cols = ['order_id','order_date','logistic_order_id','action','facility_id','facility_type','city_id',
                 'logistic_company_id','timestamp']
order_cols = ['day','order_id','item_det_info','pay_timestamp','buyer_id','promise_speed','if_cainiao','merchant_id',
              'Logistics_review_score']
seller_cols = ['day','merchant_id','subcategory_id','pc_pv','pc_uv','app_pv','app_uv','avg_logistic_review_score',
              'avg_order_quality_score','avg_service_quality_score','if_cainiao']

# print(s'=== inventory ===')
# inventory = pd.read_csv('msom_inventory_data.csv', header=None)
# inventory.columns = inventory_cols
# inventory.shape
# inventory.head()


# print('=== item ===')
# item = pd.read_csv('msom_item_data.csv', header=None) 
# item.columns = item_cols
# item.shape
# item.head()


print('=== order ===')
order = pd.read_csv('msom_order_data_1.csv', header=None)
order.columns = order_cols
order = order.drop(['day','item_det_info','buyer_id','promise_speed','if_cainiao','merchant_id'], axis=1)
order = order[~order['Logistics_review_score'].isnull()]
# for i in tqdm(range(2, 8)):
#     order_ = pd.read_csv(f'msom_order_data_{i}.csv', header=None)
#     order_.columns = order_cols
#     order_ = order_.drop(['day','item_det_info','buyer_id','promise_speed','if_cainiao','merchant_id'], axis=1)
#     order_ = order_[~order_['Logistics_review_score'].isnull()]
#     order = pd.concat([order, order_])

order.shape
order.head()


print('=== logistic ===')
logistic = pd.read_csv('msom_logistic_detail_1.csv', header=None)
logistic.columns = logistic_cols
logistic = pd.merge(logistic, order, on='order_id', how='left')
logistic = logistic[~logistic['Logistics_review_score'].isnull()]
logistic = logistic.drop_duplicates()
logistic.shape
logistic.head()

# logistic = []
# with open('msom_logistic_detail_1.csv', encoding='utf-8') as f:
#     for line in f.readlines():
#         line = line.strip()
#         features = line.split(',')
#         logistic.append(features)
# logistic = pd.DataFrame(logistic)

def importdata_logistic(filename):
    logistic = pd.read_csv(filename, header=None) 
    logistic.columns = logistic_cols
    logistic = logistic.drop(['logistic_order_id','facility_id','facility_type','city_id','logistic_company_id'], axis=1)
    logistic.shape
    logistic.head()
    return logistic


# stats_logistic = []
# print('=== stats_logistic ===')
# for i in tqdm(range(1, 8)):
#     logistic = importdata_logistic(f'msom_logistic_detail_{i}.csv')
#     logistic.shape
#     logistic = pd.merge(logistic, order, on='order_id', how='left')
#     logistic.shape
#     logistic = logistic[~logistic['Logistics_review_score'].isnull()]
#     logistic = logistic.drop_duplicates()
#     stats_logistic_ = logistic.groupby(['Logistics_review_score']).size().reset_index()
#     stats_logistic_
#     stats_logistic.append(stats_logistic_)
#     del logistic
#     gc.collect()


# print('=== seller ===')
# seller = pd.read_csv('msom_seller_data.csv', header=None) 
# seller.columns = seller_cols
# seller.shape
# seller.head()




=== order ===


KeyboardInterrupt: 

In [4]:
review1_oids = np.unique(random.sample(logistic[logistic['Logistics_review_score']==1].order_id.unique().tolist(), 10000))
review2_oids = np.unique(random.sample(logistic[logistic['Logistics_review_score']==2].order_id.unique().tolist(), 10000))
review3_oids = np.unique(random.sample(logistic[logistic['Logistics_review_score']==3].order_id.unique().tolist(), 10000))
review4_oids = np.unique(random.sample(logistic[logistic['Logistics_review_score']==4].order_id.unique().tolist(), 10000))
review5_oids = np.unique(random.sample(logistic[logistic['Logistics_review_score']==5].order_id.unique().tolist(), 10000))

# flag1_oids = logistic[logistic['Logistics_review_score']>=4].order_id.unique()
# flag0_oids = logistic[logistic['Logistics_review_score']<4].order_id.unique()

# logistic[logistic['Logistics_review_score']==5].order_id.unique()
# logistic[logistic['order_id']==114956315].sort_values('timestamp')

sample_oids = np.concatenate([review1_oids, review2_oids, review3_oids, review4_oids, review5_oids])
logistic_ = logistic[logistic['order_id'].isin(sample_oids)]
del logistic
gc.collect()

logistic = logistic_.copy()
del logistic_
gc.collect()



20

0

### (A) cut the duration into 1000th quantiles

In [ ]:
sent_times = []
for oid in tqdm(np.unique(random.sample(logistic.order_id.unique().tolist(), 10000))):
    tmp = logistic[logistic['order_id']==oid]
    reviewscore = tmp.Logistics_review_score.values[0]
    try:
        end_time = tmp[tmp['action']=='SIGNED'].timestamp.values[-1]
    except:
        pass
    start_time = tmp.pay_timestamp.values[0]
    duration = dt.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S') - dt.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    sent_times.append([oid, reviewscore, duration.total_seconds()])

df_sent_times = pd.DataFrame(sent_times, columns=['order_id','Logistics_review_score','sent_duration'])
df_sent_times


In [ ]:
# logistic.head()
# logistic[logistic['action']=='SIGNED']
logistic[logistic['order_id']==296540].sort_values('timestamp')
oid = 296540
duration_cut = 1000
tmp = logistic[logistic['order_id']==296540].sort_values('timestamp')
order_time = tmp.pay_timestamp.values[0]
reviewscore = tmp.Logistics_review_score.values[0]

end_time = tmp[tmp['action']=='SIGNED'].timestamp.values[-1]
start_time = tmp[tmp['action']=='CONSIGN'].timestamp.values[0]
duration = dt.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S') - dt.datetime.strptime(order_time, '%Y-%m-%d %H:%M:%S')

percentile_times = []
for i in range(0, duration_cut+1):
    tile = dt.datetime.strptime(order_time, '%Y-%m-%d %H:%M:%S') + ((duration/duration_cut)*i)
    percentile_times.append(dt.datetime.strftime(tile, '%Y-%m-%d %H:%M:%S'))

percentile_actions = [oid]
percentile_actions.append('ORDERED')

for j in range(0, duration_cut):
    df_duration = tmp[(tmp['timestamp']>=percentile_times[j]) & (tmp['timestamp']<=percentile_times[j+1])]
    
    if len(df_duration) > 0:
        if df_duration.action.values[-1] not in ['FAILURE','TRADE_SUCCESS']:
            percentile_actions.append(df_duration.action.values[-1])
        else:
            percentile_actions.append(percentile_actions[j+1])
    else:
        if j==0:
            percentile_actions.append('ORDERED')
        else:
            percentile_actions.append(percentile_actions[j+1])
        

percentile_actions.append(reviewscore)
percentile_actions


In [ ]:
# start: CONSIGN
# end: SIGNED

# oids = np.concatenate([np.unique(random.sample(review1_oids.tolist(), 200)), 
#                        np.unique(random.sample(review2_oids.tolist(), 200)),
#                        np.unique(random.sample(review3_oids.tolist(), 200)),
#                        np.unique(random.sample(review4_oids.tolist(), 200)), 
#                        np.unique(random.sample(review5_oids.tolist(), 200))])

# oids = np.concatenate([np.unique(random.sample(flag1_oids.tolist(), 200)), 
#                        np.unique(random.sample(flag0_oids.tolist(), 200))])

oids = np.concatenate([np.unique(random.sample(review1_oids.tolist(), 300)), 
                       np.unique(random.sample(review5_oids.tolist(), 300))])

percentile_actions_collect = []
oid_invalid = []
duration_cut = 1000

for oid in tqdm(oids):
    
    tmp = logistic[logistic['order_id']==oid].sort_values('timestamp')
    order_time = tmp.pay_timestamp.values[0]
    reviewscore = tmp.Logistics_review_score.values[0]
    try:
        end_time = tmp[tmp['action']=='SIGNED'].timestamp.values[-1]
    except:
        print(f'oid: {oid} lack of SIGNED action.')
        oid_invalid.append(oid)
    
    try:
        start_time = tmp[tmp['action']=='CONSIGN'].timestamp.values[0]
    except:
        print(f'oid: {oid} lack of CONSIGN action.')
        oid_invalid.append(oid)


    duration = dt.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S') - dt.datetime.strptime(order_time, '%Y-%m-%d %H:%M:%S')
    percentile_times = []
    for i in range(0, duration_cut+1):
        tile = dt.datetime.strptime(order_time, '%Y-%m-%d %H:%M:%S') + ((duration/duration_cut)*i)
        percentile_times.append(dt.datetime.strftime(tile, '%Y-%m-%d %H:%M:%S'))
    
    percentile_actions = [oid]
    percentile_actions.append('ORDERED')
    
    for j in range(0, duration_cut):
        df_duration = tmp[(tmp['timestamp']>=percentile_times[j]) & (tmp['timestamp']<=percentile_times[j+1])]
        
        if len(df_duration) > 0:
            if df_duration.action.values[-1] not in ['FAILURE','TRADE_SUCCESS']:
                percentile_actions.append(df_duration.action.values[-1])
            else:
                percentile_actions.append(percentile_actions[j+1])
                
        else:
            if j==0:
                percentile_actions.append('ORDERED')
            else:
                percentile_actions.append(percentile_actions[j+1])

    
    percentile_actions.append(reviewscore)
    if oid not in oid_invalid:
        percentile_actions_collect.append(percentile_actions)
    


### (B) Different Length of Sequences, just use original order

In [ ]:
# start: PAY_TIMESTAMP
# end: SIGNED

oids = np.concatenate([np.unique(random.sample(review1_oids.tolist(), 300)), 
                       np.unique(random.sample(review5_oids.tolist(), 300))])

actions_collect = []
oid_invalid = []

for oid in tqdm(oids):
    
    tmp = logistic[logistic['order_id']==oid].sort_values('timestamp')
    reviewscore = tmp.Logistics_review_score.values[0]
    
    if len(tmp[tmp['action']=='SIGNED'])==0 or len(tmp[tmp['action']=='CONSIGN'])==0:
        print(f'oid: {oid} lack of CONSIGN or SIGNED action.')
        oid_invalid.append(oid)
        

    actions = [oid, reviewscore]
    for item in tmp.action.values:
        actions.append(item)
    
    if oid not in oid_invalid:
        actions_collect.append(actions)



### (C) 以 action 間的時間間隔大小決定重複幾次，控制 sequence 長度在 100-150

In [14]:
power = 100
tmp = logistic[logistic['order_id']==logistic.order_id.values[0]].sort_values('timestamp')
order_time = tmp.pay_timestamp.values[0]
end_time = tmp[tmp['action']=='SIGNED'].timestamp.values[-1]
tmp = tmp[(tmp['timestamp']>=order_time) & (logistic['timestamp']<=end_time)]
reviewscore = tmp.Logistics_review_score.values[0]
actions = ['ORDERED'] + tmp.action.values.tolist()
actions
timestamps = np.array([dt.datetime.strptime(order_time, '%Y-%m-%d %H:%M:%S')] + [dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in tmp.timestamp.values])
gaps = timestamps[1:] - timestamps[:-1]
gaps = [x.total_seconds() for x in gaps]
gaps_percent = np.array([x/sum(gaps) for x in gaps])
gaps_percent
gaps = [int(round(x*int(power))) for x in gaps_percent]
gaps

actions_list = []
for i in range(0, len(gaps)):
    for num in range(0, gaps[i]+1):
        actions_list.append(actions[i])
actions_list.append(actions[-1])

actions_list


['ORDERED',
 'CONSIGN',
 'ARRIVAL',
 'GOT',
 'ARRIVAL',
 'DEPARTURE',
 'ARRIVAL',
 'SENT_SCAN',
 'SIGNED']

array([0.064, 0.3  , 0.   , 0.185, 0.034, 0.352, 0.   , 0.065])

[6, 30, 0, 18, 3, 35, 0, 6]

['ORDERED',
 'ORDERED',
 'ORDERED',
 'ORDERED',
 'ORDERED',
 'ORDERED',
 'ORDERED',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'CONSIGN',
 'ARRIVAL',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'GOT',
 'ARRIVAL',
 'ARRIVAL',
 'ARRIVAL',
 'ARRIVAL',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTURE',
 'DEPARTUR

In [ ]:
# start: ORDERED
# end: SIGNED

# oids = np.concatenate([np.unique(random.sample(review1_oids.tolist(), 200)), 
#                        np.unique(random.sample(review2_oids.tolist(), 200)),
#                        np.unique(random.sample(review3_oids.tolist(), 200)),
#                        np.unique(random.sample(review4_oids.tolist(), 200)), 
#                        np.unique(random.sample(review5_oids.tolist(), 200))])


oids = np.concatenate([np.unique(random.sample(review1_oids.tolist(), 300)), 
                       np.unique(random.sample(review5_oids.tolist(), 300))])

power = 100
actions_collect = []
# oid_invalid = []

for oid in tqdm(oids):
    
    tmp = logistic[logistic['order_id']==oid].sort_values('timestamp')
    order_time = tmp.pay_timestamp.values[0]
    reviewscore = tmp.Logistics_review_score.values[0]
    actions = ['ORDERED'] + tmp.action.values.tolist()
    timestamps = np.array([dt.datetime.strptime(order_time, '%Y-%m-%d %H:%M:%S')] + [dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in tmp.timestamp.values])
    gaps = timestamps[1:] - timestamps[:-1]
    
    gaps = [x.total_seconds() for x in gaps]
    gaps_percent = np.array([x/sum(gaps) for x in gaps])
    gaps = [int(round(x*int(power))) for x in gaps_percent]
    
    
#     try:
#         end_time = tmp[tmp['action']=='SIGNED'].timestamp.values[-1]
#     except:
#         print(f'oid: {oid} lack of SIGNED action.')
#         oid_invalid.append(oid)
    
#     try:
#         start_time = tmp[tmp['action']=='CONSIGN'].timestamp.values[0]
#     except:
#         print(f'oid: {oid} lack of CONSIGN action.')
#         oid_invalid.append(oid)

    
    actions_list = [oid, reviewscore]
    for i in range(0, len(gaps)):
        for num in range(0, gaps[i]+1):
            actions_list.append(actions[i])
            
    actions_list.append(actions[-1])
    actions_collect.append(actions_list)

#     if oid not in oid_invalid:
#         percentile_actions_collect.append(percentile_actions)



### (C-2) 總運送天數

In [5]:
sent_times = []
for oid in tqdm(logistic.order_id.unique()):
    tmp = logistic[logistic['order_id']==oid]
    reviewscore = tmp.Logistics_review_score.values[0]
    try:
        end_time = tmp[tmp['action']=='SIGNED'].timestamp.values[-1]
    except:
        pass
    start_time = tmp.pay_timestamp.values[0]
    duration = dt.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S') - dt.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    sent_times.append([oid, reviewscore, duration.days])

df_sent_times = pd.DataFrame(sent_times, columns=['order_id','Logistics_review_score','sent_duration'])
df_sent_times = df_sent_times[df_sent_times['sent_duration']>=0]
df_sent_times['sent_duration'] = np.where(df_sent_times['sent_duration']>=10, '10+', df_sent_times['sent_duration'])
df_sent_times.groupby(['sent_duration']).size().reset_index().rename(columns={0: 'count'})

sent_duration_0day = df_sent_times[df_sent_times['sent_duration']=='0'].order_id.unique(); print(sent_duration_0day.shape)
sent_duration_1day = df_sent_times[df_sent_times['sent_duration']=='1'].order_id.unique(); print(sent_duration_1day.shape)
sent_duration_2day = df_sent_times[df_sent_times['sent_duration']=='2'].order_id.unique(); print(sent_duration_2day.shape)
sent_duration_3day = df_sent_times[df_sent_times['sent_duration']=='3'].order_id.unique(); print(sent_duration_3day.shape)
sent_duration_4day = df_sent_times[df_sent_times['sent_duration']=='4'].order_id.unique(); print(sent_duration_4day.shape)
sent_duration_5day = df_sent_times[df_sent_times['sent_duration']=='5'].order_id.unique(); print(sent_duration_5day.shape)
sent_duration_6day = df_sent_times[df_sent_times['sent_duration']=='6'].order_id.unique(); print(sent_duration_6day.shape)
sent_duration_7day = df_sent_times[df_sent_times['sent_duration']=='7'].order_id.unique(); print(sent_duration_7day.shape)
sent_duration_8day = df_sent_times[df_sent_times['sent_duration']=='8'].order_id.unique(); print(sent_duration_8day.shape)
sent_duration_9day = df_sent_times[df_sent_times['sent_duration']=='9'].order_id.unique(); print(sent_duration_9day.shape)
sent_duration_10dayup = df_sent_times[df_sent_times['sent_duration']=='10+'].order_id.unique(); print(sent_duration_10dayup.shape)



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [01:15<00:00, 658.06it/s]


,sent_duration,count
0,0,1277
1,1,6099
2,10+,3557
3,2,9778
4,3,9411
5,4,7058
6,5,4773
7,6,3107
8,7,2194
9,8,1506


(1277,)
(6099,)
(9778,)
(9411,)
(7058,)
(4773,)
(3107,)
(2194,)
(1506,)
(1093,)
(3557,)


In [65]:
# start: ORDERED
# end: SIGNED

sent_duration_xday = sent_duration_2day
review1_oids = logistic[(logistic['order_id'].isin(sent_duration_xday)) & (logistic['Logistics_review_score']==1)].order_id.unique()
review5_oids = logistic[(logistic['order_id'].isin(sent_duration_xday)) & (logistic['Logistics_review_score']==5)].order_id.unique()
print(review1_oids.shape); print(review5_oids.shape)

oids = np.concatenate([np.unique(random.sample(review1_oids.tolist(), min(len(review1_oids), len(review5_oids), 1000))), 
                       np.unique(random.sample(review5_oids.tolist(), min(len(review1_oids), len(review5_oids), 1000)))])


power = 100
actions_collect = []
reviews_collect = []
length_count = []
oid_invalid = []

for oid in tqdm(oids):
    
    tmp = logistic[logistic['order_id']==oid].sort_values('timestamp')
    order_time = tmp.pay_timestamp.values[0]
    
    try:
        end_time = tmp[tmp['action']=='SIGNED'].timestamp.values[-1]
    except:
        print(f'oid: {oid} lack of SIGNED action.')
        oid_invalid.append(oid)
        
    tmp = tmp[(tmp['timestamp']>=order_time) & (logistic['timestamp']<=end_time)]
    
    try:
        reviewscore = tmp.Logistics_review_score.values[0]
    except:
        print(f'sequence timing is weird so that len(tmp) == 0.')
        oid_invalid.append(oid)
        
    actions = ['ORDERED'] + tmp.action.values.tolist()
    timestamps = np.array([dt.datetime.strptime(order_time, '%Y-%m-%d %H:%M:%S')] + [dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in tmp.timestamp.values])
    gaps = timestamps[1:] - timestamps[:-1]
    gaps = [x.total_seconds() for x in gaps]
    gaps_percent = np.array([x/sum(gaps) for x in gaps])
    gaps = [int(round(x*int(power))) for x in gaps_percent]
    length_count.append(len(gaps))

    
    actions_list = [oid]
    for i in range(0, len(gaps)):
        for num in range(0, gaps[i]+1):
            actions_list.append(actions[i])
            
    actions_list.append(actions[-1])
    actions_collect.append(actions_list)
    reviews_collect.append(f'reviewscore_{reviewscore}')


#     if oid not in oid_invalid:
#         percentile_actions_collect.append(percentile_actions)



  0%|                                                                                                                                                                                                              | 0/2000 [00:00<?, ?it/s]

(1520,)
(2417,)
oid: 31708 lack of SIGNED action.


  1%|█▏                                                                                                                                                                                                   | 12/2000 [00:06<16:40,  1.99it/s]

oid: 1828311 lack of SIGNED action.


  1%|█▎                                                                                                                                                                                                   | 13/2000 [00:06<16:34,  2.00it/s]

...

 98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌    | 1954/2000 [16:16<00:22,  2.01it/s]

oid: 130381462 lack of SIGNED action.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1999/2000 [16:38<00:00,  2.02it/s]

oid: 136487405 lack of SIGNED action.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [16:38<00:00,  2.00it/s]


In [66]:
max(length_count)
# percentile_actions_collect[0]
# actions_collect[0]
# pd.DataFrame(percentile_actions_collect)
pd.DataFrame(actions_collect).head()


22

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123
0,31708,ORDERED,CONSIGN,CONSIGN,GOT,GOT,GOT,GOT,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,ARRIVAL,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,None,None,None,None,None,None,None,None,None,None,None
1,180550,ORDERED,ORDERED,ORDERED,ORDERED,CONSIGN,CONSIGN,CONSIGN,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SIGNED,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,401952,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,CONSIGN,CONSIGN,CONSIGN,GOT,GOT,GOT,GOT,GOT,GOT,GOT,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,ARRIVAL,ARRIVAL,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,TRADE_SUCCESS,SIGNED,None,None,None,None,None,None,None,None,None,None
3,487025,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,GOT,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN

#### versions
**action as states**
- v1: 每個分階都抽 200 人，使用原始 review_score
- v2: 每個分階都抽 200 人，≥4分 flag=1, ≤3分 flag=0
- v2.1: 每個分階都抽 200 人，≥4分 flag=1, ≤3分 flag=0；調整states為6個
- v2.2: ≥4分 flag=1, ≤3分 flag=0；調整states為6個；兩flag各抽1000人
- v3: 區間切成 1000 等分，盡可能捕捉最後時間間隔很近的序列
- v3.1: 只抽1分, 5分，增加分群分得出來的可能性
- v4: 起始時間從 CONSIGN 改成 order 成立時間
- v5: 直接用原始順序，不切 percentiles（sequence 長度不一）
- v6: 以 action 間的時間間隔大小決定重複幾次，控制 sequence 長度在 100-150 (flag)
- v6.1: 以 action 間的時間間隔大小決定重複幾次，控制 sequence 長度在 100-150 (review1-5)

**action_count as states**
- v7: 只抽1分, 5分，固定區間數量，用區間內的action數作為state
- v8.0: 分不同運送總天數 (0天內送達，從下單到SIGNED)
- v8.1: 分不同運送總天數 (1天內送達) 
~ 
- v8.10 以此類推

**action as states**
- v9.0: 分不同運送總天數 (0天內送達，從下單到SIGNED)
- v9.1: 分不同運送總天數 (1天內送達) 
~ 
- v9.10 以此類推


In [67]:
# percentile_cols = ['percentile_'+str(x)+'th' for x in range(0, duration_cut+1)]
# percentile_cols = ['order_id'] + percentile_cols + ['review_score']

# v5, v6, v9
action_cols = ['action_'+str(x) for x in range(0, pd.DataFrame(actions_collect).shape[1]-1)]
action_cols = ['order_id'] + action_cols

order_logistic_states = pd.DataFrame(actions_collect, columns=action_cols)
order_logistic_states['review_score'] = reviews_collect
# order_logistic_states['flag'] = np.where(order_logistic_states['review_score']>=4, 1, 0)  # ≥ 4分 flag=1, 反之 flag=0
order_logistic_states.groupby(['review_score']).size()
order_logistic_states.shape
order_logistic_states.head()
order_logistic_states.to_csv('order_logistic_states-20201224-v9.2-state_delay.csv', index=False)


review_score
reviewscore_1.0    1000
reviewscore_5.0    1000
dtype: int64

(2000, 125)

,order_id,action_0,action_1,action_2,action_3,action_4,action_5,action_6,action_7,action_8,action_9,action_10,action_11,action_12,action_13,action_14,action_15,action_16,action_17,action_18,action_19,action_20,action_21,action_22,action_23,action_24,action_25,action_26,action_27,action_28,action_29,action_30,action_31,action_32,action_33,action_34,action_35,action_36,action_37,action_38,action_39,action_40,action_41,action_42,action_43,action_44,action_45,action_46,action_47,action_48,action_49,action_50,action_51,action_52,action_53,action_54,action_55,action_56,action_57,action_58,action_59,action_60,action_61,action_62,action_63,action_64,action_65,action_66,action_67,action_68,action_69,action_70,action_71,action_72,action_73,action_74,action_75,action_76,action_77,action_78,action_79,action_80,action_81,action_82,action_83,action_84,action_85,action_86,action_87,action_88,action_89,action_90,action_91,action_92,action_93,action_94,action_95,action_96,action_97,action_98,action_99,action_100,action_101,action_102,action_103,action_104,action_105,action_106,action_107,action_108,action_109,action_110,action_111,action_112,action_113,action_114,action_115,action_116,action_117,action_118,action_119,action_120,action_121,action_122,review_score
0,31708,ORDERED,CONSIGN,CONSIGN,GOT,GOT,GOT,GOT,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,ARRIVAL,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,None,None,None,None,None,None,None,None,None,None,None,reviewscore_1.0
1,180550,ORDERED,ORDERED,ORDERED,ORDERED,CONSIGN,CONSIGN,CONSIGN,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,GOT,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SIGNED,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,reviewscore_1.0
2,401952,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,CONSIGN,CONSIGN,CONSIGN,GOT,GOT,GOT,GOT,GOT,GOT,GOT,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,ARRIVAL,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,ARRIVAL,ARRIVAL,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_SCAN,SENT_S

In [68]:
collect_all = list()
for l in actions_collect:
    for item in l[1:]:
        collect_all.append(item)

pd.DataFrame(collect_all, columns=['items']).groupby(['items']).size().reset_index().rename(columns={0: 'count'})


,items,count
0,ARRIVAL,22045
1,CONSIGN,26370
2,DEPARTURE,101754
3,FAILURE,157
4,GOT,17292
5,ORDERED,28977
6,SENT_SCAN,18829
7,SIGNED,2169
8,TRADE_SUCCESS,1429
